# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1uAUJGEUzfNj6OsWNAimnYCw7eKaHhMUfU1MTj9YwYw4/edit?usp=sharing), [grading rubric](https://docs.google.com/document/d/1hKuRWqFcIdhOkow3Nljcm7PXzIkoa9c_aHkMKZDxWa0/edit?usp=sharing)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_**All code below should be consider "pseudo-code" - not functional by itself, and only an outline to help you with your own approach.**_

## Project Setup

In [ ]:
# all import statements needed for the project, for example:

import math
import os
import re

import bs4
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db
import geopandas as gpd
from keplergl import KeplerGl

In [ ]:
# any constants you might need; some have been added for you, and 
# some you need to fill in

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

TAXI_ZONES_DIR = "data/taxi_zones"
TAXI_PARQUET_DIR = "data/taxi_parquet"
TAXI_ZONES_SHAPEFILE = f"{TAXI_ZONES_DIR}/taxi_zones.shp"
UBER_CSV = "data/uber_rides_sample.csv"
WEATHER_CSV_DIR = "data/weather"

CRS = 4326  # coordinate reference system

# (lat, lon)
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
LGA_BOX_COORDS = ((40.763589, -73.891745), (40.778865, -73.854838))
JFK_BOX_COORDS = ((40.639263, -73.795642), (40.651376, -73.766264))
EWR_BOX_COORDS = ((40.686794, -74.194028), (40.699680, -74.165205))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [ ]:
# Make sure the QUERY_DIRECTORY exists
try:
    os.mkdir(QUERY_DIRECTORY)
except Exception as e:
    if e.errno == 17:
        # the directory already exists
        pass
    else:
        raise

## Part 1: Data Preprocessing

### Load Taxi Zones

In [ ]:
def load_taxi_zones(shapefile):
    taxi_zones = gpd.read_file(filename=shapefile)
    taxi_zones = taxi_zones.to_crs(CRS)
    taxi_zones['longitude'] = taxi_zones.centroid.x
    taxi_zones['latitude'] = taxi_zones.centroid.y
    taxi_zones = taxi_zones[['longitude', 'latitude']]
    return taxi_zones

In [ ]:
def lookup_coords_for_taxi_zone_id(zone_loc_id, loaded_taxi_zones):
    return loaded_taxi_zones.loc[zone_loc_id, "latitude"], loaded_taxi_zones.loc[zone_loc_id, "longitude"]

### Calculate distance

In [ ]:
def calculate_distance_with_coords(from_coord, to_coord):
    pickup_lat, pickup_long = from_coord.iloc[:,0], from_coord.iloc[:,1]
    dropoff_lat, dropoff_long = to_coord.iloc[:,0], to_coord.iloc[:,1]    
    return 12742 * (((((dropoff_lat - pickup_lat)/2).map(math.sin))**2 + (pickup_lat.map(math.cos)) * (dropoff_lat.map(math.cos)) * (((dropoff_long - pickup_long)/2).map(math.sin))**2)**0.5).map(math.asin)

In [ ]:
def calculate_distance_with_zones(from_zone, to_zone):
    raise NotImplementedError()

In [ ]:
def add_distance_column(dataframe):
    dataframe['distance'] = calculate_distance_with_coords(dataframe[['pickup_latitude', 'pickup_longitude']], dataframe[['dropoff_latitude', 'dropoff_longitude']])
    return dataframe

### Process Taxi Data

In [ ]:
def get_all_urls_from_taxi_page(taxi_page):
    response = requests.get(TAXI_URL)
    soup = bs4.BeautifulSoup(response.text, "html.parser")
    urls = soup.select('a[title="Yellow Taxi Trip Records"]')
    return urls

In [ ]:
def filter_taxi_parquet_urls(all_urls):
    pattern = r"2009|201[0-4]|2015-0[1-6]"
    parquet_urls = []
    
    for url in all_urls:
        href = url["href"]
        if re.search(pattern, href):
            parquet_urls.append(href)
        
    return parquet_urls

In [ ]:
def get_and_clean_month(url, loaded_taxi_zones):
    fname = url[url.rfind("/")+1:]
    fpath = os.path.join(TAXI_PARQUET_DIR, fname)
    if os.path.exists(fpath):
        df=pd.read_parquet(fpath)
    else:
        df = pd.read_parquet(url)
    
    if 'DOLocationID' in df:
        df['pickup_latitude'], df['pickup_longitude'] = lookup_coords_for_taxi_zone_id(df['PULocationID'], loaded_taxi_zones)
        df['dropoff_latitude'], df['dropoff_longitude']= lookup_coords_for_taxi_zone_id(df['DOLocationID'], loaded_taxi_zones)
    
    df.rename(columns={'tpep_pickup_datetime':'pickup_datetime' ,'Start_Lon':'pickup_longitude', 'Start_Lat':'pickup_latitude', 'End_Lon':'dropoff_longitude', 'End_Lat':'dropoff_latitude','Tip_Amt':'tip_amount'}, inplace=True)
    df = df[['pickup_datetime', 'pickup_latitude','pickup_longitude', 'dropoff_latitude','dropoff_longitude', 'tip_amount']]
        
    lower_lat, lower_long = NEW_YORK_BOX_COORDS[0]
    upper_lat, upper_long = NEW_YORK_BOX_COORDS[1]
    
    df = df[(df['tip_amount'] >= 0)]
    
    df = df[(df['pickup_latitude'] >= lower_lat) & (df['pickup_latitude'] <= upper_lat)]
    df = df[(df['pickup_longitude'] >= lower_long) & (df['pickup_longitude'] <= upper_long)]
    df = df[(df['dropoff_latitude'] >= lower_lat) & (df['dropoff_latitude'] <= upper_lat)]
    df = df[(df['dropoff_longitude'] >= lower_long) & (df['dropoff_longitude'] <= upper_long)]

    df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
    df["pickup_latitude"] = pd.to_numeric(df["pickup_latitude"], errors = "coerce")
    df["pickup_longitude"] = pd.to_numeric(df["pickup_longitude"], errors = "coerce")
    df["dropoff_latitude"] = pd.to_numeric(df["dropoff_latitude"], errors = "coerce")
    df["dropoff_longitude"] = pd.to_numeric(df["dropoff_longitude"], errors = "coerce")
    df["tip_amount"] = pd.to_numeric(df["tip_amount"], errors = "coerce")
    
    df = df.dropna().sort_values(by="pickup_datetime")
    df.index = range(df.shape[0])
    
    return df

In [ ]:
def get_and_clean_taxi_data(parquet_urls):
    all_taxi_dataframes = []
    loaded_taxi_zones = load_taxi_zones(TAXI_ZONES_SHAPEFILE)
    
    for parquet_url in parquet_urls:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_month(parquet_url, loaded_taxi_zones)
        add_distance_column(dataframe)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        
        all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.contact(all_taxi_dataframes)
    return taxi_data

In [ ]:
def get_taxi_data():
    all_urls = get_all_urls_from_taxi_page(TAXI_URL)
    all_parquet_urls = filter_taxi_parquet_urls(all_urls)
    taxi_data = get_and_clean_taxi_data(all_parquet_urls)
    return taxi_data

In [ ]:
taxi_data = get_taxi_data()

In [ ]:
taxi_data.head()

### Processing Uber Data

In [ ]:
def load_and_clean_uber_data(csv_file):
    df = pd.read_csv(UBER_CSV, index_col=[0])
    
    df = df[['pickup_datetime', 'pickup_latitude','pickup_longitude', 'dropoff_latitude','dropoff_longitude']]
        
    lower_lat, lower_long = NEW_YORK_BOX_COORDS[0]
    upper_lat, upper_long = NEW_YORK_BOX_COORDS[1]
    
    df = df[(df['pickup_latitude'] >= lower_lat) & (df['pickup_latitude'] <= upper_lat)]
    df = df[(df['pickup_longitude'] >= lower_long) & (df['pickup_longitude'] <= upper_long)]
    df = df[(df['dropoff_latitude'] >= lower_lat) & (df['dropoff_latitude'] <= upper_lat)]
    df = df[(df['dropoff_longitude'] >= lower_long) & (df['dropoff_longitude'] <= upper_long)]

    df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
    df["pickup_latitude"] = pd.to_numeric(df["pickup_latitude"], errors = "coerce")
    df["pickup_longitude"] = pd.to_numeric(df["pickup_longitude"], errors = "coerce")
    df["dropoff_latitude"] = pd.to_numeric(df["dropoff_latitude"], errors = "coerce")
    df["dropoff_longitude"] = pd.to_numeric(df["dropoff_longitude"], errors = "coerce")
    
    df = df.dropna().sort_values(by="pickup_datetime")
    df.index = range(df.shape[0])
    
    return df

In [ ]:
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_CSV)
    add_distance_column(uber_dataframe)
    return uber_dataframe

In [ ]:
uber_data = get_uber_data()

In [ ]:
uber_data.head()

### Processing Weather Data

In [ ]:
def get_all_weather_csvs(directory):
    filenames = os.listdir(directory)
    return [os.path.join(directory, filename) for filename in filenames]

In [ ]:
def clean_month_weather_data_hourly(csv_file):
    df = pd.read_csv(csv_file, index_col=[0], low_memory=False)
    df = df[["DATE","HourlyWindSpeed","HourlyPrecipitation"]]

    df["DATE"] = pd.to_datetime(df["DATE"])
    df["HourlyPrecipitation"] = pd.to_numeric(df["HourlyPrecipitation"], errors = "coerce")
    df["HourlyWindSpeed"] = pd.to_numeric(df["HourlyWindSpeed"], errors = "coerce")
        
    df = df.rename(columns={"DATE": "weather_date", "HourlyWindSpeed":"wind_speed", "HourlyPrecipitation":"precipitation"})
    df = df.dropna().sort_values(by="weather_date")
    df.index = range(df.shape[0])
    
    return df

In [ ]:
def clean_month_weather_data_daily(csv_file):
    hourly_df = clean_month_weather_data_hourly(csv_file)
    hourly_df['weather_date'] = hourly_df['weather_date'].dt.date
    daily_df = hourly_df.groupby(["weather_date"], as_index = False).agg({"precipitation":"sum","wind_speed":"mean"})
    daily_df['weather_date'] = pd.to_datetime(daily_df['weather_date'])
    daily_df = daily_df.dropna().sort_values(by="weather_date")
    daily_df.index = range(daily_df.shape[0])
    return daily_df

In [ ]:
def load_and_clean_weather_data():
    weather_csv_files = get_all_weather_csvs(WEATHER_CSV_DIR)
    
    hourly_dataframes = []
    daily_dataframes = []
        
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

In [ ]:
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

In [ ]:
hourly_weather_data.head()

In [ ]:
daily_weather_data.head()

## Part 2: Storing Cleaned Data
take the sample datasets generated from Part 1, and populating a SQLite database with tables generated from the datasets


In [ ]:
engine = db.create_engine(DATABASE_URL)

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    weather_date DATE,
    wind_speed FLOAT,
    precipitation FLOAT
);
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    weather_date DATE,
    wind_speed FLOAT,
    precipitation FLOAT
);
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    pickup_datetime DATE,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_datetime DATE,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT,
    tip_amount FLOAT,
    distance FLOAT
);
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    pickup_datetime DATE,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT,
    distance FLOAT
);
"""

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [ ]:
# create the tables with the schema files
from sqlalchemy import text
with engine.connect() as connection:
    for query in [HOURLY_WEATHER_SCHEMA, DAILY_WEATHER_SCHEMA, TAXI_TRIPS_SCHEMA, UBER_TRIPS_SCHEMA]:
        connection.execute(query)

### Add Data to Database

In [ ]:
def write_dataframes_to_table(table_to_df_dict):
    for tab in table_to_df_dict:
        table_to_df_dict[tab].to_sql(tab, engine, if_exists='append', index=False)

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_weather_data,
    "daily_weather": daily_weather_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    raise NotImplementedError()

### Query 1

In [ ]:
QUERY_1_FILENAME = ""

QUERY_1 = """
TODO
"""

In [ ]:
engine.execute(QUERY_1).fetchall()

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)